In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import imageio
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames[:10]:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"

In [ ]:
!pip install "../input/hpapytorchzoozip/pytorch_zoo-master"

In [ ]:
!pip install "../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"

In [ ]:
def build_image_names(image_id: str) -> list:
    # mt is the mitchondria
    mt = f'/kaggle/input/hpa-single-cell-image-classification/test/{image_id}_red.png'
    
    # er is the endoplasmic reticulum
    er = f'/kaggle/input/hpa-single-cell-image-classification/test/{image_id}_yellow.png'
    
    # nu is the nuclei
    nu = f'/kaggle/input/hpa-single-cell-image-classification/test/{image_id}_blue.png'
    
    return [mt], [er], [nu], [[mt], [er], [nu]]

In [ ]:
mt, er, nu, images = build_image_names(image_id='277b3f6d-099b-4b6d-8592-a06ad6f52beb')

In [ ]:
images

In [ ]:
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei

NUC_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
CELL_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'

segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device='cuda',
    padding=False,
    multi_channel_model=True
)

In [ ]:
# For nuclei
nuc_segmentations = segmentator.pred_nuclei(images[2])

# For full cells
cell_segmentations = segmentator.pred_cells(images)

In [ ]:
# post-processing
i = 0
nuclei_mask, cell_mask = label_cell(nuc_segmentations[i], cell_segmentations[i])

In [ ]:
import matplotlib.pyplot as plt

i = 0
plt.figure(figsize=(20, 10))
microtubule = plt.imread(mt[i])    
endoplasmicrec = plt.imread(er[i])    
nuclei = plt.imread(nu[i])
mask = cell_mask
img = np.dstack((microtubule, endoplasmicrec, nuclei))
plt.imshow(img)
plt.imshow(mask, alpha=0.6)
plt.show()

In [ ]:
# Plot of just the mask
plt.imshow(mask, alpha=0.6)